# CC5215: Privacidad de Datos

## Laboratorio 5

Integrantes:

- Francisco Gutiérrez Albornoz

In [26]:
# Load the data and libraries
import pandas as pd
import numpy as np
import random
from scipy import stats
import matplotlib.pyplot as plt
plt.style.use('seaborn-v0_8-whitegrid')

def laplace_mech(v, sensitivity, epsilon):
    return v + np.random.laplace(loc=0, scale=sensitivity / epsilon)

def pct_error(orig, priv):
    return np.abs(orig - priv)/orig * 100.0

adult = pd.read_csv('https://users.dcc.uchile.cl/~mtoro/cursos/cc5215/adult_with_pii.csv')


**Note**: this function (and the rest of the ones you'll define in this assignment) take a list of *query results* rather than the queries themselves (as we saw in class). This simplification makes your code a little bit simpler.

In [27]:
# preserves epsilon-differential privacy
def above_threshold(query_results, T, epsilon):
    T_hat = T + np.random.laplace(loc=0, scale = 2/epsilon)

    for idx, q in enumerate(query_results):
        nu_i = np.random.laplace(loc=0, scale = 4/epsilon)
        if q + nu_i >= T_hat:
            return idx
    return None

## Question 1 (5 points)

The code below defines a finite set of options for marital status. Define a scoring function that returns high scores for common marital statuses, and low scores for uncommon ones (e.g. the score could be the number of people with that status). Remember that your function should be 1-sensitive.

In [28]:
options = ['Never-married', 'Married-civ-spouse', 'Divorced',
           'Married-spouse-absent', 'Separated', 'Married-AF-spouse',
           'Widowed']

def score(option):

    return adult['Marital Status'].value_counts()[option]/1000

score('Never-married')

np.float64(10.683)

In [29]:
assert score('Never-married') > score('Divorced')
assert score('Never-married') > score('Widowed')
assert score('Never-married') < score('Married-civ-spouse')
assert score('Never-married') > score('Divorced')

## Question 2 (10 points)

Implement `report_noisy_max` using the Laplace mechanism. `report_noisy_max` should return the value in a set that approximately maximizes the value of the score function. It should not return the score itself.

In [30]:
def report_noisy_max(R, score, sensitivity, epsilon):
    scores = [score(x) for x in R]
    noisy_scores = [laplace_mech(s, sensitivity, epsilon) for s in scores]
    max_idx = np.argmax(noisy_scores)
    return R[max_idx]

report_noisy_max(options, score, 1, 1)

'Married-civ-spouse'

In [31]:
assert report_noisy_max(options, score, 1, 1) == 'Married-civ-spouse'

## Question 3 (5 points)

Implement a function `above_10000` that releases the **value** of the first query in a sequence of queries whose value is above 10000. Your function should have a **total** privacy cost equal to the privacy parameter $\epsilon$ passed in when it is called.

In [32]:
def above_10000(query_results, epsilon):
    T_hat = 10000 + np.random.laplace(loc=0, scale=2/epsilon)
    for idx, q in enumerate(query_results):
        nu_i = np.random.laplace(loc=0, scale=4/epsilon)
        if q + nu_i >= T_hat:
            return query_results.iloc[idx]
    return None

queries = adult['Marital Status'].value_counts()
print(f"above_10000 #1: {above_10000(queries, 100)}")
print(f"above_10000 #2: {above_10000(queries, 1)}")
print(f"above_10000 #3: {above_10000(queries, .01)}")

above_10000 #1: 14976
above_10000 #2: 14976
above_10000 #3: 14976


In [33]:
# TEST CASE

results = [above_10000(queries, 1.0) for _ in range(20)]
print(np.mean(results))
assert np.mean(results) > 14900
assert np.mean(results) < 15000

14976.0


## Question 4 (5 points)
In 2-3 sentences, argue informally that your implementation of `above_10000` has a total privacy cost of $\epsilon$.

*Respuesta*: En clases no se ahondó en la demostración de esto pero se utiliza `epsilon / 2` en fijar un threshold (ruidoso). Luego, se compara el umbral con cada query lo que tiene un coste de `epsilon / 4`, en el peor caso para `n` queries se tiene `n * epsilon / 4` (por composición secuencial). Está demostrado que este algoritmo es `epsilon-DP` (pero no se vio a detalle).

## Question 5 (10 points)

Implement a function `bounded_all_above_10000` that releases the **value** of **$c$ queries** in a sequence of queries whose value is above 10000 (where $c$ is an analyst-provided parameter limiting the number of returned results). Your function should have a **total privacy cost** bounded by its parameter $\epsilon$.

In [38]:
def bounded_all_above_10000(query_results, c, epsilon):
    def index_above_10000(query_results, epsilon):
        T_hat = 10000 + np.random.laplace(loc=0, scale=2/epsilon)
        for idx, q in enumerate(query_results):
            nu_i = np.random.laplace(loc=0, scale=4/epsilon)
            if q + nu_i >= T_hat:
                return idx
        return None
    idxs = []
    pos = 0
    epsilon_i = epsilon / c
    while pos < len(query_results) and len(idxs) < c:
        next_idx = index_above_10000(query_results[pos:], epsilon_i)
        if next_idx == None:
            return [query_results[i] for i in idxs]
        pos = next_idx + pos
        idxs.append(pos)
        pos += 1
    return [query_results[i] for i in idxs]

queries = list(adult['Marital Status'].value_counts())
print(f"bounded_all_above_10000 #1: {bounded_all_above_10000(queries, 3, 100)}")
print(f"bounded_all_above_10000 #2: {bounded_all_above_10000(queries, 3, 1)}")
print(f"bounded_all_above_10000 #3: {bounded_all_above_10000(queries, 3, .01)}")

bounded_all_above_10000 #1: [14976, 10683]
bounded_all_above_10000 #2: [14976, 10683]
bounded_all_above_10000 #3: [14976]


In [39]:
# TEST CASE

results = [bounded_all_above_10000(queries, 2, 1.0)]
results_1 = [r[0] for r in results]
results_2 = [r[1] for r in results]

assert np.mean(results_1) > 14900
assert np.mean(results_1) < 15000
assert np.mean(results_2) > 10600
assert np.mean(results_2) < 10700

## Question 6 (5 points)

In 2-3 sentences, argue informally that your implementation of `bounded_all_above_10000` has privacy cost bounded by $\epsilon$.

*Respuesta*: Cada invocación para encontrar una query que cumple con el threshold se utiliza `epsilon / c` como se buscan `c` queries que cumplan con el threshold, por composición secuencial se utiliza a lo más `epsilon` (es a lo más ya que podrían haber menos de `c` queries que lo cumplan).

## Question 7 (15 points)

Implement a function `mean_age` that computes the mean age of participants in the `adult_data` dataset. Your function should have a **total** privacy cost of $\epsilon$. It should work as follows:

1. Compute an *upper* clipping parameter based on the data
2. Clip the data using the clipping parameter
3. Use `laplace_mech` to release a differentially private mean of the clipped data

*Hint*: Use the sparse vector technique (`above_threshold`) to compute the clipping parameter. Consider using a sequence of queries that looks like `df.clip(lower=b, upper=0).sum() - df.clip(lower=b+1, upper=0).sum()`.

*Hint*: Be careful of sensitivities and set the scale of the noise accordingly!

In [40]:
bs = list(range(0, 200, 10))
df = adult['Age']

def mean_age(epsilon):
    def create_query(b):
        return lambda df: df.clip(lower=0, upper=b).sum() - df.clip(lower=0, upper=b + 1).sum()
    def above_threshold(queries, df, T, epsilon):
        T_hat = T + np.random.laplace(loc=0, scale=2/epsilon)
        for idx, q in enumerate(queries):
            nu_i = np.random.laplace(loc=0, scale=4/epsilon)
            if q(df) + nu_i >= T_hat:
                return idx
        return None
    queries_age = [create_query(b) for b in bs]
    epsilon_svt = epsilon / 3
    final_b = bs[above_threshold(queries_age, df, 0, epsilon_svt)]

    epsilon_sum = epsilon / 3
    epsilon_count = epsilon / 3

    noisy_sum = laplace_mech(df.clip(lower=0, upper=final_b).sum(), final_b, epsilon_sum)
    noisy_count = laplace_mech(len(df), 1, epsilon_count)
    return noisy_sum / noisy_count

for epsilon in [0.001, 0.01, 0.1, 0.5, 1, 10]:
    print(f"epsilon: {epsilon}, mean age: {mean_age(epsilon)}")

epsilon: 0.001, mean age: 21.259463062853786
epsilon: 0.01, mean age: 40.44376037039085
epsilon: 0.1, mean age: 38.47423904109511
epsilon: 0.5, mean age: 38.60297598478245
epsilon: 1, mean age: 38.58358623161321
epsilon: 10, mean age: 38.58341987136645


In [41]:
# TEST CASE
results = [mean_age(1.0) for _ in range(20)]
assert np.mean(results) > 38
assert np.mean(results) < 39

## Question 8 (5 points)

In 3-5 sentences, describe your approach to implementing `mean_age` and argue informally that your implementation has privacy cost bounded by $\epsilon$.

*Respuesta*: Seguí casi exactamente lo que se hizo en clases, se crean queries que sean 1-sensitiva (esto se consigue con la resta). Luego, se utiliza `epsilon / 3` para encontrar el parámetro de clipping `b` adecuado utilizando SVT mediante la función `above_threshold`. Finalmente, se calcula suma y conteo con ruido en donde para cada una se utiliza `epsilon / 3` y por composición secuencial se tiene que es `epsilon-DP`.  